![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


In [5]:
# Import necessary packages
import pandas as pd
import torch

from transformers import logging
logging.set_verbosity(logging.WARNING)


In [6]:
# Start your code here!
path = "data/car_reviews.csv"
df = pd.read_csv(path, delimiter=";")

reviews = df['Review'].tolist()
classes = df['Class'].tolist()

from transformers import pipeline
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
predicted_labels = classifier(reviews)
print(predicted_labels)

import evaluate
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

references = [1 if label == "POSITIVE" else 0 for label in classes]
predictions = [1 if label['label'] == "POSITIVE" else 0 for label in predicted_labels]

accuracy_result_dict = accuracy.compute(references=references, predictions=predictions)
accuracy_result = accuracy_result_dict['accuracy']
f1_result_dict = f1.compute(references=references, predictions=predictions)
f1_result = f1_result_dict['f1']
print(f"Accuracy: {accuracy_result}")
print(f"F1 result: {f1_result}")

################################################# 2 #######################################################

first_review = reviews[0]
sentences = first_review.split('.')
first_two_sentences = '.'.join(sentences[:2]).strip() + '.'
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
translated_review = translator(first_two_sentences)[0]['translation_text']
print(f"Model translation:\n{translated_review}")

with open("data/reference_translations.txt", 'r') as file:
    lines = file.readlines()
references = [line.strip() for line in lines]
print(f"Spanish translation references:\n{references}")

bleu = evaluate.load("bleu")
bleu_score = bleu.compute(predictions=[translated_review], references=[references])
print(bleu_score['bleu'])

################################################# 3 #######################################################

from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
qa_model = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(qa_model)
model = AutoModelForQuestionAnswering.from_pretrained(qa_model)

context = reviews[1]
question = "What did he like about the brand?"
inputs = tokenizer(question, context, return_tensors="pt")

with torch.no_grad():
  outputs = model(**inputs)
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]

answer = tokenizer.decode(answer_span)
print("Extractive QA answer:", answer)
################################################# 4 #######################################################

# Get original text to summarize upon car review
text_to_summarize = reviews[-1]
print(f"Original text:\n{text_to_summarize}")

# Load summarization pipeline and perform inference
model_name = "cnicu/t5-small-booksum"
summarizer = pipeline("summarization", model=model_name)
outputs = summarizer(text_to_summarize, max_length=53)
summarized_text = outputs[0]['summary_text']
print(f"Summarized text:\n{summarized_text}")

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.929397702217102}, {'label': 'POSITIVE', 'score': 0.8654273152351379}, {'label': 'POSITIVE', 'score': 0.9994640946388245}, {'label': 'NEGATIVE', 'score': 0.9935314059257507}, {'label': 'POSITIVE', 'score': 0.9986565113067627}]
Accuracy: 0.8
F1 result: 0.8571428571428571


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Device set to use cpu


Summarized text:
the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. I have hauled 12 bags of mulch in the back with the seats down and could have held more.
